In [13]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime, date
from sklearn.preprocessing import *
import openpyxl

In [2]:
dataset = pd.read_excel('./data/carddata3.xlsx', engine="openpyxl")

In [3]:
dataset.columns=["APPR_NO", "MCC_CODE", "MCC_NAME", "ACQ_CLASS", "APPR_DATE", "APPR_TIME", "MERCH_ADDR", "MERCH_NAME", "ABOARD", "APPR_AMT","TIPS","ACQU_FEE", "VAT", "ETC1", "ETC3", "CRD_FG_CD", "CRD_STL_DD", "EMP_NO", "EMP_DV_CD", "GRADE_CD", "TITLE_CD", "BP_CD", "BA_CD", "HKONT", "ACCT_NM"]
dataset = dataset.astype({'MCC_CODE': 'object', 'ETC3': 'object', 'TITLE_CD':'object', 'CRD_STL_DD':'object','BA_CD':'object' ,'HKONT': 'object'})
dataset = dataset.dropna(axis=0).reset_index()

for index, row in dataset.iterrows():
    if dataset["MERCH_ADDR"][index].isalpha():
        dataset["MERCH_ADDR"][index] = "Foreign"
    else:
        dataset["MERCH_ADDR"][index] = dataset["MERCH_ADDR"][index].split(" ")[1]

In [4]:
new_data = pd.DataFrame()
label_onehot = pd.DataFrame()
label_label = pd.DataFrame()
cate_col = ["MCC_CODE","MERCH_ADDR","ABOARD","ETC1","ETC3","CRD_FG_CD","CRD_STL_DD","TITLE_CD","BA_CD", "HKONT"]
day_week_array = []
day_range_array = []
time_range_array = []

# 금액 데이터 처리
appr_amt = np.array(dataset["APPR_AMT"]).reshape(-1, 1)
vat = np.array(dataset["VAT"]).reshape(-1, 1)
ss_amt=StandardScaler().fit_transform(appr_amt)
ss_vat=StandardScaler().fit_transform(vat)
reg_amt=MinMaxScaler().fit_transform(appr_amt)
reg_vat=MinMaxScaler().fit_transform(vat)
ss_amt_frame = pd.DataFrame(ss_amt)
ss_vat_frame = pd.DataFrame(ss_vat)
reg_amt_frame = pd.DataFrame(reg_amt)
reg_vat_frame = pd.DataFrame(reg_vat)

new_data = pd.concat([new_data, ss_amt_frame], axis=1)
new_data = pd.concat([new_data, ss_vat_frame], axis=1)
#new_data = pd.concat([new_data, reg_amt_frame], axis=1)
#new_data = pd.concat([new_data, reg_vat_frame], axis=1)
#new_data = pd.concat([new_data, dataset["APPR_AMT"]], axis=1)
#new_data = pd.concat([new_data, dataset["VAT"]], axis=1)

# 날짜 데이터 처리
datatemp = pd.DataFrame(dataset["APPR_DATE"])
for index, row in datatemp.iterrows():
    temp_year  = int(str(dataset["APPR_DATE"][index])[0:4])
    temp_month = int(str(dataset["APPR_DATE"][index])[4:6])
    temp_day   = int(str(dataset["APPR_DATE"][index])[6:8])
    day = date(temp_year, temp_month, temp_day).weekday()

    if temp_day < 10:
        range_day = 0
    elif temp_day >= 10 and temp_day < 20:
        range_day = 1
    else:
        range_day = 2    
        
    day_week_array.append(day)
    day_range_array.append(range_day)

new_data = pd.concat([new_data, pd.DataFrame(day_week_array)], axis=1)
new_data = pd.concat([new_data, pd.DataFrame(day_range_array)], axis=1)

# 시간 데이터 처리
datatemp = pd.DataFrame(dataset["APPR_TIME"])
for index, row in datatemp.iterrows():
    temp_time   = int(str(dataset["APPR_TIME"][index])[0:2])

    if temp_time <10:
        range_time = 0
    elif temp_time >= 10 and temp_time < 14:
        range_time = 1
    elif temp_time >= 14 and temp_time < 18:
        range_time = 2
    else:
        range_time = 3

    time_range_array.append(range_time)    
    
new_data = pd.concat([new_data, pd.DataFrame(time_range_array)], axis=1)    

# categorical data 처리
for col in dataset.columns:
    if col in cate_col:
        if col == 'HKONT':
            le = LabelEncoder()
            le.fit(dataset.HKONT)
            temp = pd.DataFrame(le.transform(dataset.HKONT))
            label_label = pd.concat([label_label, temp], axis = 1)
            
            enc = OneHotEncoder(handle_unknown ='ignore')
            enc.fit(np.array(dataset[col]).reshape(-1,1))
            temp = pd.DataFrame(enc.transform(np.array(dataset[col]).reshape(-1,1)).toarray())
            label_onehot = pd.concat([label_onehot, temp], axis = 1)
        else:
            enc = OneHotEncoder(handle_unknown ='ignore')
            # col_name = enc.get_feature_names([col])
            # encoded = enc.fit_transform(dataset[col])
            # one_hot_encoded_frame = pd.DataFrame(encoded ,columns=col_name)
            enc.fit(np.array(dataset[col]).reshape(-1,1))
            temp = pd.DataFrame(enc.transform(np.array(dataset[col]).reshape(-1,1)).toarray())
            new_data = pd.concat([new_data, temp], axis=1)

In [5]:
new_col_index = np.array(range(0,len(new_data.T)),int)
new_data.columns = new_col_index
new_data

,0,1,2,3,4,5,6,7,8,9,...,337,338,339,340,341,342,343,344,345,346
0,0.214297,0.236965,4,2,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2.449873,2.551062,6,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.248730,0.272536,4,2,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.093402,-0.081406,0,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.298192,0.324030,3,1,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37209,-0.289216,-0.283953,2,2,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37210,-0.301264,-0.296447,1,2,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37211,-0.298253,-0.293311,0,2,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37212,-0.301264,-0.296447,0,2,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
label_label

,0
0,4
1,4
2,4
3,1
4,4
...,...
37209,12
37210,12
37211,12
37212,12


In [7]:
data_set = pd.concat([new_data,label_label], axis=1)
data_set

,0,1,2,3,4,5,6,7,8,9,...,338,339,340,341,342,343,344,345,346,0
0,0.214297,0.236965,4,2,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
1,2.449873,2.551062,6,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
2,0.248730,0.272536,4,2,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
3,-0.093402,-0.081406,0,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,0.298192,0.324030,3,1,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37209,-0.289216,-0.283953,2,2,2,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
37210,-0.301264,-0.296447,1,2,1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
37211,-0.298253,-0.293311,0,2,1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
37212,-0.301264,-0.296447,0,2,1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12


In [9]:
count = label_label.sum()
label_num = label_label.value_counts()
label_label.value_counts()

0     11764
1      9477
9      4855
8      3864
5      2005
4      1429
10     1169
11      873
12      686
7       403
6       381
3       166
2       142
dtype: int64

In [8]:
data_set.to_csv('./data/pre_data.csv', index=False)